============================================================
### 02 - NETTOYAGE DES DONNÉES
============================================================

**Objectif** : Traiter les anomalies identifiées lors de l'AED
- Suppression des valeurs impossibles
- Imputation des valeurs manquantes
- Gestion des outliers

In [146]:
# ============= IMPORTS =============
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import os
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import joblib

In [97]:
# Configuration
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [98]:
df = pd.read_csv('../data/donnees_recrutement.csv')

In [99]:
# Suppression de la colonne Unnamed: 0
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)

#### SECTION 1 : NETTOYAGE DES DONNÉES

##### 1.1 TRAITEMENT DES VALEURS IMPOSSIBLES

**detections des valeurs impossibles**

In [100]:
# --- AGE : détection ---
print("\n📌 Variable AGE :")
print("-" * 40)

# Critères pour âge impossible/anormal
age_negatif = df['age'] < 0
age_trop_jeune = (df['age'] >= 0) & (df['age'] < 15)


print(f"   • Âges NÉGATIFS (< 0) : {age_negatif.sum()}")
print(f"   • Âges TROP JEUNES (0-14) : {age_trop_jeune.sum()}")

print(f"   • TOTAL anomalies AGE : {(age_negatif | age_trop_jeune ).sum()}")


📌 Variable AGE :
----------------------------------------
   • Âges NÉGATIFS (< 0) : 3
   • Âges TROP JEUNES (0-14) : 354
   • TOTAL anomalies AGE : 357


In [101]:
if age_negatif.sum() > 0:
    print(f"\n   cas avec age négative :")
    exemples_exp = df[age_negatif][['age', 'exp', 'diplome', 'salaire', 'specialite']].head(10)
    print(exemples_exp.to_string())


   cas avec age négative :
       age   exp   diplome  salaire specialite
1842  -1.0   6.0       bac  38176.0  detective
3968  -3.0  11.0  doctorat  26167.0     forage
19605 -2.0  12.0  doctorat  27837.0   geologie


In [102]:
if age_trop_jeune.sum() > 0:
    print(f"\n   cas avec age négative :")
    exemples_exp = df[age_trop_jeune][['age', 'exp', 'diplome', 'salaire', 'specialite']].head(10)
    print(exemples_exp.to_string())


   cas avec age négative :
      age   exp   diplome  salaire   specialite
112  14.0  13.0    master  34851.0       forage
164  10.0   9.0    master  38767.0  archeologie
211  12.0  10.0    master  31557.0    detective
219  14.0   9.0   licence  32178.0    detective
231  13.0  17.0  doctorat  29115.0     geologie
258   8.0  10.0   licence  44530.0  archeologie
270  13.0   7.0       bac  30800.0    detective
290  14.0   7.0    master  41795.0       forage
296   8.0  16.0    master  25825.0  archeologie
396   8.0  11.0    master  36915.0     geologie


In [103]:
# --- EXP : détection ---
print("\n\n📌 Variable EXP :")
print("-" * 40)

exp_negatif = df['exp'] < 0

print(f"   • Expériences NÉGATIVES (< 0) : {exp_negatif.sum()}")




📌 Variable EXP :
----------------------------------------
   • Expériences NÉGATIVES (< 0) : 2


In [104]:
if exp_negatif.sum() > 0:
    print(f"\n   Cas avec EXP négative :")
    exemples_exp = df[exp_negatif][['age', 'exp', 'diplome', 'salaire', 'specialite']].head(10)
    print(exemples_exp.to_string())


   Cas avec EXP négative :
        age  exp  diplome  salaire specialite
6025   36.0 -1.0   master  29241.0   geologie
11284  34.0 -2.0  licence  51294.0  detective


**Imputations des ces valeurs**

In [105]:
def imputer_par_profils_similaires(df, index, variable_a_imputer, tolerance_salaire=5000):
    """
    Impute AGE ou EXP en se basant sur des profils similaires (diplôme + salaire)
    
    Paramètres :
    - df : DataFrame
    - index : index de la ligne à imputer
    - variable_a_imputer : 'age' ou 'exp'
    - tolerance_salaire : écart acceptable pour le salaire (défaut: 5000)
    
    Retourne :
    - valeur imputée (ou None si impossible)
    """
    
    # Récupérer diplôme et salaire du candidat
    diplome_cible = df.loc[index, 'diplome']
    salaire_cible = df.loc[index, 'salaire']
    specialite_cible=df.loc[index,'specialite']
    
    # Si salaire manquant, impossible d'utiliser cette méthode
    if pd.isna(salaire_cible):
        return None
    
    # Trouver les profils similaires
    mask = (
        (df['diplome'] == diplome_cible) &  # Même diplôme
        (df['specialite'] == specialite_cible) &
        (df['salaire'] >= salaire_cible - tolerance_salaire) &  # Salaire proche
        (df['salaire'] <= salaire_cible + tolerance_salaire) &
        (df[variable_a_imputer] > 0) &  # Valeur valide
        (df.index != index)  # Exclure la ligne elle-même
    )
    
    profils_similaires = df[mask]
    
    # Si aucun profil trouvé, retourner None
    if len(profils_similaires) == 0:
        return None
    
    # Retourner la médiane des profils similaires
    return profils_similaires[variable_a_imputer].median()

In [106]:
print("\n🔧 CORRECTION des AGE négatifs :\n")

nb_corriges = 0
nb_echecs = 0

for idx in df[age_negatif].index:
    age_original = df.loc[idx, 'age']
    
    # Tenter l'imputation
    valeur_imputee = imputer_par_profils_similaires(df, idx, 'age')
    
    if valeur_imputee is not None:
        df.loc[idx, 'age'] = valeur_imputee
        nb_corriges += 1
        print(f"   Index {idx}: {age_original:.1f} → {valeur_imputee:.1f}")
    else:
        # Fallback: médiane du diplôme
        diplome = df.loc[idx, 'diplome']
        mean_diplome = df[(df['diplome'] == diplome) & (df['age'] > 0)]['age'].mean()
        df.loc[idx, 'age'] = mean_diplome
        nb_echecs += 1
        print(f"   Index {idx}: {age_original:.1f} → {mean_diplome:.1f} (fallback)")

print(f"\n   Corrigés par profils similaires : {nb_corriges}")
print(f"   Corrigés par fallback : {nb_echecs}")



🔧 CORRECTION des AGE négatifs :

   Index 1842: -1.0 → 36.0
   Index 3968: -3.0 → 35.0
   Index 19605: -2.0 → 35.0

   Corrigés par profils similaires : 3
   Corrigés par fallback : 0


In [107]:
# =============================================================================
# APPLICATION - EXP NÉGATIFS
# =============================================================================

print("\n🔧 CORRECTION des EXP négatifs :\n")

nb_corriges = 0
nb_echecs = 0

for idx in df[exp_negatif].index:
    exp_original = df.loc[idx, 'exp']
    
    # Tenter l'imputation
    valeur_imputee = imputer_par_profils_similaires(df, idx, 'exp')
    
    if valeur_imputee is not None:
        df.loc[idx, 'exp'] = valeur_imputee
        nb_corriges += 1
        print(f"   Index {idx}: {exp_original:.1f} → {valeur_imputee:.1f}")
    else:
        # Fallback: médiane du diplôme
        diplome = df.loc[idx, 'diplome']
        mean_diplome = df[(df['diplome'] == diplome) & (df['exp'] >= 0)]['exp'].mean()
        df.loc[idx, 'exp'] = mean_diplome
        nb_echecs += 1
        print(f"   Index {idx}: {exp_original:.1f} → {mean_diplome:.1f} (fallback)")

print(f"\n   Corrigés par profils similaires : {nb_corriges}")
print(f"   Corrigés par fallback : {nb_echecs}")


🔧 CORRECTION des EXP négatifs :

   Index 6025: -1.0 → 10.0
   Index 11284: -2.0 → 9.0

   Corrigés par profils similaires : 2
   Corrigés par fallback : 0


##### 1.2 TRAITEMENT DES VALEURS MANQUANTES

**Detections des valeurs manquantes**

In [108]:

print("\n" + "="*80)
print("DÉTECTION DES VALEURS MANQUANTES")
print("="*80)

# Calculer le nombre de valeurs manquantes par variable
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
total_missing = df.isnull().sum().sum()
total_cells = df.shape[0] * df.shape[1]
missing_percent = (total_missing / total_cells) * 100


# Créer un DataFrame récapitulatif
missing_df = pd.DataFrame({
    'Variable': missing.index,
    'Nb_manquants': missing.values,
    'Pourcentage': missing_pct.values
})

# Filtrer uniquement les variables avec valeurs manquantes
missing_df = missing_df[missing_df['Nb_manquants'] > 0].sort_values('Nb_manquants', ascending=False)

print("\nRésumé des valeurs manquantes :\n")

if len(missing_df) > 0:
    print(missing_df.to_string(index=False))
    print(f"\n   Total de valeurs manquantes : {missing_df['Nb_manquants'].sum()} ({missing_percent:.3f}% du dataset)")

else:
    print("    Aucune valeur manquante détectée !")
    print("\n   → Rien à faire, passage à l'étape suivante.")
    exit()


DÉTECTION DES VALEURS MANQUANTES

Résumé des valeurs manquantes :

  Variable  Nb_manquants  Pourcentage
      note           114        0.570
   diplome           110        0.550
     dispo           106        0.530
   cheveux           103        0.515
      sexe           100        0.500
       exp            96        0.480
   salaire            95        0.475
specialite            93        0.465
      date            91        0.455
       age            91        0.455

   Total de valeurs manquantes : 999 (0.416% du dataset)


In [109]:
print("\n" + "="*80)
print("IDENTIFICATION DES TYPES DE VARIABLES")
print("="*80)

# Variables numériques et catégorielles
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Retirer la variable cible et la date des listes
if 'embauche' in numeric_cols:
    numeric_cols.remove('embauche')
if 'date' in categorical_cols:
    categorical_cols.remove('date')

print(f"\nVariables numériques ({len(numeric_cols)}) : {numeric_cols}")
print(f"Variables catégorielles ({len(categorical_cols)}) : {categorical_cols}")


IDENTIFICATION DES TYPES DE VARIABLES

Variables numériques (5) : ['index', 'age', 'exp', 'salaire', 'note']
Variables catégorielles (5) : ['cheveux', 'sexe', 'diplome', 'specialite', 'dispo']


**STRATÉGIE 1 : SUPPRESSION DES VALEURS MANQUANTES**

Justification de la suppression : total valeurs maquantes < 1% 

In [110]:
df_dropna = df.dropna().copy()

print(f"\n Résultats :")
print(f"   • Lignes avant suppression : {len(df)}")
print(f"   • Lignes après suppression : {len(df_dropna)}")
print(f"   • Lignes supprimées : {len(df) - len(df_dropna)} ({(len(df) - len(df_dropna))/len(df)*100:.2f}%)")
print(f"   • Valeurs manquantes restantes : {df_dropna.isnull().sum().sum()}")

print("\nDataset 1 créé : df_dropna (suppression)")


 Résultats :
   • Lignes avant suppression : 20000
   • Lignes après suppression : 19021
   • Lignes supprimées : 979 (4.90%)
   • Valeurs manquantes restantes : 0

Dataset 1 créé : df_dropna (suppression)


**STRATÉGIE 2 : IMPUTATION MÉDIANE/MODE** 

In [111]:
df_median_mode = df.copy()

print("\nImputation des variables NUMÉRIQUES par la MÉDIANE :")
print("-" * 80)

for col in numeric_cols:
    if df_median_mode[col].isnull().sum() > 0:
        median_value = df_median_mode[col].median()
        missing_count = df_median_mode[col].isnull().sum()
        df_median_mode[col].fillna(median_value, inplace=True)
        print(f"   • {col:15s} : {missing_count:3d} valeurs imputées avec médiane = {median_value:.2f}")

print("\nImputation des variables CATÉGORIELLES par le MODE :")
print("-" * 80)

for col in categorical_cols:
    if df_median_mode[col].isnull().sum() > 0:
        mode_value = df_median_mode[col].mode()[0]
        missing_count = df_median_mode[col].isnull().sum()
        df_median_mode[col].fillna(mode_value, inplace=True)
        print(f"    {col:15s} : {missing_count:3d} valeurs imputées avec mode = '{mode_value}'")

# Traitement de la colonne date (si manquante)
if 'date' in df_median_mode.columns and df_median_mode['date'].isnull().sum() > 0:
    mode_date = df_median_mode['date'].mode()[0]
    missing_count = df_median_mode['date'].isnull().sum()
    df_median_mode['date'].fillna(mode_date, inplace=True)
    print(f"   {'date':15s} : {missing_count:3d} valeurs imputées avec mode")

print(f"\nValeurs manquantes restantes : {df_median_mode.isnull().sum().sum()}")
print(" Dataset 2 créé : df_median_mode (médiane/mode)")


Imputation des variables NUMÉRIQUES par la MÉDIANE :
--------------------------------------------------------------------------------
   • age             :  91 valeurs imputées avec médiane = 35.00
   • exp             :  96 valeurs imputées avec médiane = 9.00
   • salaire         :  95 valeurs imputées avec médiane = 34979.00
   • note            : 114 valeurs imputées avec médiane = 75.08

Imputation des variables CATÉGORIELLES par le MODE :
--------------------------------------------------------------------------------
    cheveux         : 103 valeurs imputées avec mode = 'chatain'
    sexe            : 100 valeurs imputées avec mode = 'M'
    diplome         : 110 valeurs imputées avec mode = 'master'
    specialite      :  93 valeurs imputées avec mode = 'geologie'
    dispo           : 106 valeurs imputées avec mode = 'non'
   date            :  91 valeurs imputées avec mode

Valeurs manquantes restantes : 0
 Dataset 2 créé : df_median_mode (médiane/mode)


**STRATÉGIE 3 : IMPUTATION PAR KNN**

In [113]:
df_knn = df.copy()

# Pour KNN, on va séparer les colonnes catégorielles et numériques
print("\nÉtape 1 : Préparation des données pour KNN")
print("-" * 80)

# Créer une copie pour le travail
df_for_knn = df_knn.copy()

# Dictionnaire pour stocker les mappings
col_mappings = {}

# Encoder les colonnes catégorielles
for col in categorical_cols:
    if col in df_for_knn.columns:
        # Créer un mapping personnalisé
        unique_vals = df_for_knn[col].dropna().unique()
        mapping = {val: idx for idx, val in enumerate(unique_vals)}
        mapping[np.nan] = np.nan  # Garder les NaN
        
        col_mappings[col] = {v: k for k, v in mapping.items()}  # Inverse mapping pour le décodage
        
        df_for_knn[col] = df_for_knn[col].map(mapping)
        print(f"   • {col:15s} encodé ({len(unique_vals)} catégories)")

# Traiter la date
if 'date' in df_for_knn.columns:
    df_for_knn['date'] = pd.to_datetime(df_for_knn['date'], errors='coerce')
    df_for_knn['date_numeric'] = df_for_knn['date'].astype('int64') / 10**9
    df_for_knn = df_for_knn.drop('date', axis=1)
    print(f"   • {'date':15s} convertie en timestamp")

# Séparer la variable cible
embauche_col = df_for_knn['embauche'].copy()
df_for_knn = df_for_knn.drop('embauche', axis=1)


Étape 1 : Préparation des données pour KNN
--------------------------------------------------------------------------------
   • cheveux         encodé (4 catégories)
   • sexe            encodé (2 catégories)
   • diplome         encodé (4 catégories)
   • specialite      encodé (4 catégories)
   • dispo           encodé (2 catégories)
   • date            convertie en timestamp


In [114]:
print("\n Étape 2 : Application de KNNImputer")
print("-" * 80)

# Initialiser KNNImputer
knn_imputer = KNNImputer(n_neighbors=5, weights='uniform')

print(f"   • Paramètres : n_neighbors=5, weights='uniform'")
print(f"   • Variables à imputer : {df_for_knn.shape[1]}")

# Appliquer KNN
df_imputed_knn = pd.DataFrame(
    knn_imputer.fit_transform(df_for_knn),
    columns=df_for_knn.columns,
    index=df_for_knn.index
)

print(f" Imputation terminée")


 Étape 2 : Application de KNNImputer
--------------------------------------------------------------------------------
   • Paramètres : n_neighbors=5, weights='uniform'
   • Variables à imputer : 11
 Imputation terminée


In [115]:
print("\n Étape 3 : Décodage des variables catégorielles")
print("-" * 80)

# Décoder les variables catégorielles
for col in categorical_cols:
    if col in df_imputed_knn.columns:
        # Arrondir pour obtenir des entiers valides
        df_imputed_knn[col] = df_imputed_knn[col].round()
        
        # Récupérer le mapping inverse
        inverse_mapping = col_mappings[col]
        
        # S'assurer que les valeurs sont dans la plage valide
        valid_codes = list(inverse_mapping.keys())
        valid_codes = [x for x in valid_codes if not pd.isna(x)]
        min_code, max_code = min(valid_codes), max(valid_codes)
        df_imputed_knn[col] = df_imputed_knn[col].clip(min_code, max_code)
        
        # Décoder
        df_imputed_knn[col] = df_imputed_knn[col].map(inverse_mapping)
        
        print(f"   • {col:15s} décodé")

# Reconvertir la date
if 'date_numeric' in df_imputed_knn.columns:
    df_imputed_knn['date'] = pd.to_datetime(df_imputed_knn['date_numeric'] * 10**9, unit='ns')
    df_imputed_knn['date'] = df_imputed_knn['date'].dt.strftime('%Y-%m-%d')
    df_imputed_knn = df_imputed_knn.drop('date_numeric', axis=1)
    print(f"   • {'date':15s} reconvertie")

# Ajouter la colonne embauche
df_imputed_knn['embauche'] = embauche_col

# Réorganiser les colonnes dans l'ordre original
df_knn = df_imputed_knn[df.columns]

print(f"\n Valeurs manquantes restantes : {df_knn.isnull().sum().sum()}")
print("Dataset 3 créé : df_knn (KNN imputation)")


 Étape 3 : Décodage des variables catégorielles
--------------------------------------------------------------------------------
   • cheveux         décodé
   • sexe            décodé
   • diplome         décodé
   • specialite      décodé
   • dispo           décodé
   • date            reconvertie

 Valeurs manquantes restantes : 91
Dataset 3 créé : df_knn (KNN imputation)


In [116]:
print("\nDistribution de la variable cible 'embauche' (vérification) :")
print("="*80)

embauche_comparison = pd.DataFrame({
    'Stratégie': ['Suppression', 'Médiane/Mode', 'KNN'],
    'Non embauchés (0)': [
        (df_dropna['embauche'] == 0).sum(),
        (df_median_mode['embauche'] == 0).sum(),
        (df_knn['embauche'] == 0).sum()
    ],
    'Embauchés (1)': [
        (df_dropna['embauche'] == 1).sum(),
        (df_median_mode['embauche'] == 1).sum(),
        (df_knn['embauche'] == 1).sum()
    ]
})

embauche_comparison['Ratio (0/1)'] = (embauche_comparison['Non embauchés (0)'] / 
                                       embauche_comparison['Embauchés (1)']).round(2)

print(embauche_comparison.to_string(index=False))


Distribution de la variable cible 'embauche' (vérification) :
   Stratégie  Non embauchés (0)  Embauchés (1)  Ratio (0/1)
 Suppression              16842           2179         7.73
Médiane/Mode              17708           2292         7.73
         KNN              17708           2292         7.73


In [118]:
# ============= SAUVEGARDE DES 3 DATASETS =============
print("\n" + "="*80)
print("SAUVEGARDE DES 3 DATASETS PRÉPARÉS")
print("="*80)

# Sauvegarder les datasets
df_dropna.to_csv('../data/df_dropna.csv', index=False)
df_median_mode.to_csv('../data/df_median_mode.csv', index=False)
df_knn.to_csv('../data/df_knn.csv', index=False)

print("\nDatasets sauvegardés :")
print(f"   • df_dropna.csv      : {df_dropna.shape[0]} lignes × {df_dropna.shape[1]} colonnes")
print(f"   • df_median_mode.csv : {df_median_mode.shape[0]} lignes × {df_median_mode.shape[1]} colonnes")
print(f"   • df_knn.csv         : {df_knn.shape[0]} lignes × {df_knn.shape[1]} colonnes")



SAUVEGARDE DES 3 DATASETS PRÉPARÉS

Datasets sauvegardés :
   • df_dropna.csv      : 19021 lignes × 12 colonnes
   • df_median_mode.csv : 20000 lignes × 12 colonnes
   • df_knn.csv         : 20000 lignes × 12 colonnes


##### 1.3 GESTION DES OUTLIERS

**Detectiions des outliers**

In [122]:
# ============= FONCTION DE DÉTECTION DES OUTLIERS =============

def detect_outliers_iqr(df, column):
    """
    Détecte les outliers avec la méthode IQR (Interquartile Range)
    Outliers = valeurs < Q1 - 1.5*IQR OU > Q3 + 1.5*IQR
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    
    return {
        'Q1': Q1,
        'Q3': Q3,
        'IQR': IQR,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound,
        'n_outliers': len(outliers),
        'pct_outliers': (len(outliers) / len(df)) * 100,
        'outlier_indices': outliers.index.tolist()
    }

def detect_outliers_zscore(df, column, threshold=3):
    """
    Détecte les outliers avec la méthode Z-score
    Outliers = |Z-score| > threshold (généralement 3)
    """
    z_scores = np.abs(stats.zscore(df[column].dropna()))
    outliers_mask = z_scores > threshold
    n_outliers = outliers_mask.sum()
    
    return {
        'threshold': threshold,
        'n_outliers': n_outliers,
        'pct_outliers': (n_outliers / len(df)) * 100
    }

In [123]:
# ============= ANALYSE DES OUTLIERS SUR df_dropna (RÉFÉRENCE) =============
print("\n" + "="*80)
print("IDENTIFICATION DES OUTLIERS - Dataset de référence (df_dropna)")
print("="*80)

outliers_summary = []

for col in numeric_cols:
    print(f"\n{'='*80}")
    print(f" Variable : {col.upper()}")
    print(f"{'='*80}")
    
    # Méthode IQR
    iqr_results = detect_outliers_iqr(df_dropna, col)
    
    # Méthode Z-score
    zscore_results = detect_outliers_zscore(df_dropna, col, threshold=3)
    
    print(f"\nMéthode IQR (Interquartile Range) :")
    print(f"   • Q1 (25e percentile)     : {iqr_results['Q1']:.2f}")
    print(f"   • Q3 (75e percentile)     : {iqr_results['Q3']:.2f}")
    print(f"   • IQR (Q3 - Q1)           : {iqr_results['IQR']:.2f}")
    print(f"   • Limite inférieure       : {iqr_results['lower_bound']:.2f}")
    print(f"   • Limite supérieure       : {iqr_results['upper_bound']:.2f}")
    print(f"   • Nombre d'outliers       : {iqr_results['n_outliers']} ({iqr_results['pct_outliers']:.2f}%)")
    
    print(f"\nMéthode Z-Score (threshold = 3) :")
    print(f"   • Nombre d'outliers       : {zscore_results['n_outliers']} ({zscore_results['pct_outliers']:.2f}%)")
    
    # Statistiques descriptives
    print(f"\nStatistiques descriptives :")
    print(f"   • Moyenne                 : {df_dropna[col].mean():.2f}")
    print(f"   • Médiane                 : {df_dropna[col].median():.2f}")
    print(f"   • Écart-type              : {df_dropna[col].std():.2f}")
    print(f"   • Min                     : {df_dropna[col].min():.2f}")
    print(f"   • Max                     : {df_dropna[col].max():.2f}")
    
    # Évaluation de la sévérité
    severity = "FAIBLE" if iqr_results['pct_outliers'] < 5 else "MODÉRÉ" if iqr_results['pct_outliers'] < 10 else "ÉLEVÉ"
    print(f"\nNiveau de présence d'outliers : {severity}")
    
    # Stocker les résultats
    outliers_summary.append({
        'Variable': col,
        'N_outliers_IQR': iqr_results['n_outliers'],
        'Pct_outliers_IQR': iqr_results['pct_outliers'],
        'N_outliers_Zscore': zscore_results['n_outliers'],
        'Pct_outliers_Zscore': zscore_results['pct_outliers'],
        'Lower_bound': iqr_results['lower_bound'],
        'Upper_bound': iqr_results['upper_bound']
    })


IDENTIFICATION DES OUTLIERS - Dataset de référence (df_dropna)

 Variable : INDEX

Méthode IQR (Interquartile Range) :
   • Q1 (25e percentile)     : 4986.00
   • Q3 (75e percentile)     : 14993.00
   • IQR (Q3 - Q1)           : 10007.00
   • Limite inférieure       : -10024.50
   • Limite supérieure       : 30003.50
   • Nombre d'outliers       : 0 (0.00%)

Méthode Z-Score (threshold = 3) :
   • Nombre d'outliers       : 0 (0.00%)

Statistiques descriptives :
   • Moyenne                 : 9993.01
   • Médiane                 : 9989.00
   • Écart-type              : 5771.34
   • Min                     : 0.00
   • Max                     : 19999.00

Niveau de présence d'outliers : FAIBLE

 Variable : AGE

Méthode IQR (Interquartile Range) :
   • Q1 (25e percentile)     : 29.00
   • Q3 (75e percentile)     : 41.00
   • IQR (Q3 - Q1)           : 12.00
   • Limite inférieure       : 11.00
   • Limite supérieure       : 59.00
   • Nombre d'outliers       : 203 (1.07%)

Méthode Z-Score (t

In [125]:
# ============= TABLEAU RÉCAPITULATIF =============
print("\n" + "="*80)
print("TABLEAU RÉCAPITULATIF DES OUTLIERS")
print("="*80)

outliers_df = pd.DataFrame(outliers_summary)
print("\n" + outliers_df.to_string(index=False))


TABLEAU RÉCAPITULATIF DES OUTLIERS

Variable  N_outliers_IQR  Pct_outliers_IQR  N_outliers_Zscore  Pct_outliers_Zscore  Lower_bound  Upper_bound
   index               0          0.000000                  0             0.000000    -10024.50     30003.50
     age             203          1.067241                 61             0.320698        11.00        59.00
     exp               6          0.031544                 45             0.236581        -0.50        19.50
 salaire             117          0.615110                 46             0.241838     21453.00     48493.00
    note             143          0.751801                 48             0.252353        29.32       121.08



Globalement le taux d'outliers est faible pour toutes les varaibles donc on laisse comme ça.
Cependant pour les notes >100 on va faire des imputations puisque la note maximale c'est 100.


In [128]:
# =============================================================================
# FONCTION D'IMPUTATION PAR PLUS PROCHE VOISIN
# =============================================================================

def imputer_notes_knn(df_input, nom_base):
    """
    Impute les notes > 100 en utilisant la note du plus proche voisin valide
    
    Paramètres :
    - df_input : DataFrame à traiter
    - nom_base : nom du fichier pour l'affichage
    
    Retourne :
    - DataFrame avec notes imputées
    """
    
    print(f"\n{'='*80}")
    print(f"Traitement de : {nom_base}")
    print(f"{'='*80}")
    
    # Copie pour ne pas modifier l'original
    df = df_input.copy()
    
    # Identifier les notes > 100
    mask_outliers = df['note'] > 100
    nb_outliers = mask_outliers.sum()
    
    print(f"\nDétection :")
    print(f"Notes > 100 : {nb_outliers}")
    
    if nb_outliers == 0:
        print("Aucune note > 100 détectée")
        return df
    
    print(f"Pourcentage : {100*nb_outliers/len(df):.2f}%")
    
    
    # Préparer les données pour KNN
    # Encoder le diplôme
    le = LabelEncoder()
    df['diplome_encoded'] = le.fit_transform(df['diplome'])
    
    # Variables pour calculer la similarité
    features = ['age', 'exp', 'salaire', 'diplome_encoded']
    
    # Séparer les données valides (note ≤ 100) et invalides (note > 100)
    df_valides = df[df['note'] <= 100].copy()
    df_invalides = df[df['note'] > 100].copy()
    
    print(f"\nImputation par plus proche voisin :")
    print(f"Candidats valides (note ≤ 100) : {len(df_valides)}")
    print(f"Candidats à imputer (note > 100) : {len(df_invalides)}")
    
    # Créer le modèle KNN
    knn = NearestNeighbors(n_neighbors=1, metric='euclidean')
    knn.fit(df_valides[features])
    
    # Pour chaque note > 100, trouver le plus proche voisin
    corrections = []
    
    for idx in df_invalides.index:
        note_originale = df.loc[idx, 'note']
        
        # Caractéristiques du candidat
        candidat_features = df.loc[idx, features].values.reshape(1, -1)
        
        # Trouver le plus proche voisin
        distance, indice = knn.kneighbors(candidat_features)
        voisin_idx = df_valides.iloc[indice[0][0]].name
        note_voisin = df_valides.loc[voisin_idx, 'note']
        
        # Imputer
        df.loc[idx, 'note'] = note_voisin
        
        corrections.append({
            'index': idx,
            'note_originale': note_originale,
            'note_imputee': note_voisin,
            'distance': distance[0][0],
            'age': df.loc[idx, 'age'],
            'exp': df.loc[idx, 'exp'],
            'diplome': df.loc[idx, 'diplome']
        })
    
    # Afficher les corrections
    print(f"\n{len(corrections)} notes imputées")
    
    # Supprimer la colonne temporaire
    df.drop('diplome_encoded', axis=1, inplace=True)
    
    # Vérification finale
    nb_outliers_final = (df['note'] > 100).sum()
    print(f"\nVérification finale :")
    print(f"   Notes > 100 restantes : {nb_outliers_final}")
    
    if nb_outliers_final == 0:
        print(f"SUCCÈS : Toutes les notes sont maintenant ≤ 100")
    
    # Statistiques après correction
    print(f"\nStatistiques de NOTE après correction :")
    print(f"   • Min : {df['note'].min():.2f}")
    print(f"   • Max : {df['note'].max():.2f}")
    print(f"   • Moyenne : {df['note'].mean():.2f}")
    print(f"   • Médiane : {df['note'].median():.2f}")
    
    return df


In [133]:
# =============================================================================
# APPLICATION SUR LES 3 BASES DE DONNÉES
# =============================================================================

print("\n" + "="*80)
print("APPLICATION SUR LES 3 BASES DE DONNÉES")
print("="*80)

# Chemin du dossier data
data_path = '../data'

bases = [
    'df_dropna.csv',
    'df_median_mode.csv', 
    'df_knn.csv'
]

for nom_base in bases:
    try:
        # Chemin complet
        chemin_fichier = os.path.join(data_path, nom_base)
        
        # Charger la base
        df = pd.read_csv(chemin_fichier)
        
        # Appliquer l'imputation
        df_corrige = imputer_notes_knn(df, nom_base)
        
        # Sauvegarder
        df_corrige.to_csv(chemin_fichier, index=False)
        print(f"\n   Base sauvegardée : {chemin_fichier}")
        
    except FileNotFoundError:
        print(f"\n   ATTENTION : Fichier non trouvé : {chemin_fichier}")
        continue
    except Exception as e:
        print(f"\n   ERREUR lors du traitement de {nom_base} : {e}")
        continue


APPLICATION SUR LES 3 BASES DE DONNÉES

Traitement de : df_dropna.csv

Détection :
Notes > 100 : 1401
Pourcentage : 7.37%

Imputation par plus proche voisin :
Candidats valides (note ≤ 100) : 17620
Candidats à imputer (note > 100) : 1401

1401 notes imputées

Vérification finale :
   Notes > 100 restantes : 0
SUCCÈS : Toutes les notes sont maintenant ≤ 100

Statistiques de NOTE après correction :
   • Min : 8.68
   • Max : 100.00
   • Moyenne : 72.97
   • Médiane : 74.00

   Base sauvegardée : ../data\df_dropna.csv

Traitement de : df_median_mode.csv

Détection :
Notes > 100 : 1465
Pourcentage : 7.33%

Imputation par plus proche voisin :
Candidats valides (note ≤ 100) : 18535
Candidats à imputer (note > 100) : 1465

1465 notes imputées

Vérification finale :
   Notes > 100 restantes : 0
SUCCÈS : Toutes les notes sont maintenant ≤ 100

Statistiques de NOTE après correction :
   • Min : 8.68
   • Max : 100.00
   • Moyenne : 72.97
   • Médiane : 74.06

   Base sauvegardée : ../data\df_me

#### SECTION 2 : FEATURE ENGINEERING

Objectif : Créer de nouvelles variables pour améliorer le pouvoir prédictif
Stratégie : 
- Ratios et variables dérivées (exp/age, salaire/exp, etc.)
- Variables temporelles (mois, trimestre, année)
- Catégorisation de variables continues
- Interactions entre variables


In [134]:
# =============================================================================
# FONCTION DE FEATURE ENGINEERING
# =============================================================================

def creer_features(df_input, nom_base):
    """
    Crée de nouvelles features à partir des variables existantes
    
    Paramètres :
    - df_input : DataFrame à enrichir
    - nom_base : nom du fichier pour l'affichage
    
    Retourne :
    - DataFrame enrichi avec nouvelles features
    """
    
    print(f"\n{'='*80}")
    print(f"Traitement de : {nom_base}")
    print(f"{'='*80}")
    
    df = df_input.copy()
    
    print(f"\nNombre de colonnes AVANT : {len(df.columns)}")
    
    # -------------------------------------------------------------------------
    # 1. VARIABLES TEMPORELLES
    # -------------------------------------------------------------------------
    
    print("\n[1/5] Extraction des variables temporelles...")
    
    # Convertir la colonne date si nécessaire
    if df['date'].dtype == 'object':
        df['date'] = pd.to_datetime(df['date'])
    
    # Extraire les composantes temporelles
    df['annee'] = df['date'].dt.year
    df['mois'] = df['date'].dt.month
    df['trimestre'] = df['date'].dt.quarter
    df['jour_semaine'] = df['date'].dt.dayofweek  # 0=Lundi, 6=Dimanche
    df['semaine_annee'] = df['date'].dt.isocalendar().week
    
    print(f"   - 5 variables temporelles créées (annee, mois, trimestre, jour_semaine, semaine_annee)")
    
    # -------------------------------------------------------------------------
    # 2. RATIOS ET VARIABLES DÉRIVÉES
    # -------------------------------------------------------------------------
    
    print("\n[2/5] Création de ratios et variables dérivées...")
    
    # Ratio expérience / âge (expérience relative)
    df['exp_age_ratio'] = df['exp'] / (df['age'] - 18)
    df['exp_age_ratio'] = df['exp_age_ratio'].replace([np.inf, -np.inf], 0)  # Gérer division par zéro
    
    # Salaire par année d'expérience
    df['salaire_par_exp'] = df['salaire'] / (df['exp'] + 1)  # +1 pour éviter division par zéro
    
    # Différence entre expérience réelle et expérience maximale possible
    df['ecart_exp_max'] = (df['age'] - 18) - df['exp']
    
    print(f"   - 4 ratios créés (exp_age_ratio, salaire_par_exp, ecart_exp_max)")
    
    # -------------------------------------------------------------------------
    # 3. CATÉGORISATION DE VARIABLES CONTINUES
    # -------------------------------------------------------------------------
    
    print("\n[3/5] Catégorisation des variables continues...")
    
    # Catégories d'âge
    df['categorie_age'] = pd.cut(df['age'], 
                                  bins=[0, 25, 35, 45, 100], 
                                  labels=['Jeune', 'Junior', 'Confirme', 'Senior'])
    
    # Niveaux d'expérience
    df['niveau_experience'] = pd.cut(df['exp'], 
                                      bins=[-1, 2, 5, 10, 100], 
                                      labels=['Debutant', 'Intermediaire', 'Experimente', 'Expert'])
    
    # Tranches de salaire
    df['tranche_salaire'] = pd.cut(df['salaire'], 
                                    bins=[0, 30000, 35000, 40000, 100000], 
                                    labels=['Bas', 'Moyen', 'Eleve', 'Tres_eleve'])
    
    # Niveaux de note
    df['niveau_note'] = pd.cut(df['note'], 
                                bins=[0, 50, 70, 85, 100], 
                                labels=['Faible', 'Moyen', 'Bon', 'Excellent'])
    
    print(f"   - 4 catégorisations créées (categorie_age, niveau_experience, tranche_salaire, niveau_note)")
    
    # -------------------------------------------------------------------------
    # 4. INTERACTIONS ENTRE VARIABLES
    # -------------------------------------------------------------------------
    
    print("\n[4/5] Création d'interactions entre variables...")
    
    # Encoder le diplôme en numérique pour les interactions
    diplome_mapping = {'bac': 1, 'licence': 2, 'master': 3, 'doctorat': 4}
    df['diplome_num'] = df['diplome'].map(diplome_mapping)
    
    # Interaction diplôme × expérience
    df['diplome_x_exp'] = df['diplome_num'] * df['exp']
    
    # Interaction diplôme × note
    df['diplome_x_note'] = df['diplome_num'] * df['note']
    
    # Interaction expérience × note
    df['exp_x_note'] = df['exp'] * df['note']
    
    # Score composite : (diplôme + exp + note normalisée)
    df['score_composite'] = (
        df['diplome_num'] * 10 + 
        df['exp'] + 
        df['note'] / 10
    )
    
    print(f"   - 5 interactions créées (diplome_x_exp, diplome_x_note, exp_x_note, score_composite, diplome_num)")
    
    # -------------------------------------------------------------------------
    # 5. VARIABLES BINAIRES ET FLAGS
    # -------------------------------------------------------------------------
    
    print("\n[5/5] Création de variables binaires et flags...")
    
    # Flag : expérience élevée pour l'âge
    df['exp_elevee_pour_age'] = (df['exp_age_ratio'] > df['exp_age_ratio'].median()).astype(int)
    
    # Flag : salaire élevé
    df['salaire_eleve'] = (df['salaire'] > df['salaire'].median()).astype(int)
    
    # Flag : note élevée
    df['note_elevee'] = (df['note'] > df['note'].median()).astype(int)
    
    # Flag : diplôme élevé (master ou doctorat)
    df['diplome_eleve'] = df['diplome'].isin(['master', 'doctorat']).astype(int)
    
    # Flag : senior (âge > 40 et exp > 10)
    df['est_senior'] = ((df['age'] > 40) & (df['exp'] > 10)).astype(int)
    
    # Flag : profil atypique (doctorat mais salaire bas)
    df['profil_atypique'] = ((df['diplome'] == 'doctorat') & (df['salaire'] < 30000)).astype(int)
    
    print(f"   - 6 flags binaires créés (exp_elevee_pour_age, salaire_eleve, note_elevee, diplome_eleve, est_senior, profil_atypique)")
    
    # -------------------------------------------------------------------------
    # RÉCAPITULATIF
    # -------------------------------------------------------------------------
    
    print(f"\n{'='*80}")
    print(f"RÉCAPITULATIF - {nom_base}")
    print(f"{'='*80}")
    
    nb_nouvelles_features = len(df.columns) - len(df_input.columns)
    
    print(f"\nNombre de colonnes APRÈS : {len(df.columns)}")
    print(f"Nouvelles features créées : {nb_nouvelles_features}")
    
    print(f"\nCatégories de features créées :")
    print(f"   - Variables temporelles : 5")
    print(f"   - Ratios et dérivées : 4")
    print(f"   - Catégorisations : 4")
    print(f"   - Interactions : 5")
    print(f"   - Flags binaires : 6")
    print(f"   TOTAL : 24 nouvelles features")
    
    return df


In [135]:

# =============================================================================
# APPLICATION SUR LES 3 BASES DE DONNÉES
# =============================================================================

print("\n" + "="*80)
print("APPLICATION SUR LES 3 BASES DE DONNÉES")
print("="*80)

# Chemin du dossier data
data_path = '../data'

bases = [
    'df_dropna.csv',
    'df_median_mode.csv', 
    'df_knn.csv'
]

for nom_base in bases:
    try:
        # Chemin complet
        chemin_fichier = os.path.join(data_path, nom_base)
        
        # Charger la base
        print(f"\nChargement de {nom_base}...")
        df = pd.read_csv(chemin_fichier)
        
        # Appliquer le feature engineering
        df_enrichi = creer_features(df, nom_base)
        
        # Sauvegarder avec un nouveau nom pour garder trace
        nom_output = nom_base.replace('.csv', '_features.csv')
        chemin_output = os.path.join(data_path, nom_output)
        df_enrichi.to_csv(chemin_output, index=False)
        
        print(f"\nBase enrichie sauvegardée : {chemin_output}")
        
    except FileNotFoundError:
        print(f"\nATTENTION : Fichier non trouvé : {chemin_fichier}")
        continue
    except Exception as e:
        print(f"\nERREUR lors du traitement de {nom_base} : {e}")
        import traceback
        traceback.print_exc()
        continue



APPLICATION SUR LES 3 BASES DE DONNÉES

Chargement de df_dropna.csv...

Traitement de : df_dropna.csv

Nombre de colonnes AVANT : 12

[1/5] Extraction des variables temporelles...
   - 5 variables temporelles créées (annee, mois, trimestre, jour_semaine, semaine_annee)

[2/5] Création de ratios et variables dérivées...
   - 4 ratios créés (exp_age_ratio, salaire_par_exp, ecart_exp_max)

[3/5] Catégorisation des variables continues...
   - 4 catégorisations créées (categorie_age, niveau_experience, tranche_salaire, niveau_note)

[4/5] Création d'interactions entre variables...
   - 5 interactions créées (diplome_x_exp, diplome_x_note, exp_x_note, score_composite, diplome_num)

[5/5] Création de variables binaires et flags...
   - 6 flags binaires créés (exp_elevee_pour_age, salaire_eleve, note_elevee, diplome_eleve, est_senior, profil_atypique)

RÉCAPITULATIF - df_dropna.csv

Nombre de colonnes APRÈS : 35
Nouvelles features créées : 23

Catégories de features créées :
   - Variables te

In [137]:

# =============================================================================
# VÉRIFICATION DES NOUVELLES FEATURES
# =============================================================================

print("\n" + "="*80)
print("VÉRIFICATION DES NOUVELLES FEATURES")
print("="*80)

# Charger une des bases enrichies pour vérification
try:
    chemin_verif = os.path.join(data_path, 'df_median_mode_features.csv')
    df_verif = pd.read_csv(chemin_verif)
    
    print(f"\nAperçu des nouvelles features (df_median_mode_features.csv) :")
    print(f"\nColonnes ajoutées :")
    
    colonnes_originales = ['index', 'date', 'cheveux', 'age', 'exp', 'salaire', 
                          'sexe', 'diplome', 'specialite', 'note', 'dispo', 'embauche']
    nouvelles_colonnes = [col for col in df_verif.columns if col not in colonnes_originales]
    
    for i, col in enumerate(nouvelles_colonnes, 1):
        print(f"   {i}. {col}")
    
    print(f"\nStatistiques descriptives des nouvelles variables numériques :")
    nouvelles_num = df_verif[nouvelles_colonnes].select_dtypes(include=[np.number])
    print(nouvelles_num.describe().round(2))
    
    print(f"\nExemples de valeurs :")
    print(df_verif[['age', 'exp', 'exp_age_ratio', 'categorie_age', 'niveau_experience', 
                   'score_composite', 'est_senior']].head(10))

except Exception as e:
    print(f"\nImpossible de charger le fichier de vérification : {e}")


VÉRIFICATION DES NOUVELLES FEATURES

Aperçu des nouvelles features (df_median_mode_features.csv) :

Colonnes ajoutées :
   1. annee
   2. mois
   3. trimestre
   4. jour_semaine
   5. semaine_annee
   6. exp_age_ratio
   7. salaire_par_exp
   8. ecart_exp_max
   9. categorie_age
   10. niveau_experience
   11. tranche_salaire
   12. niveau_note
   13. diplome_num
   14. diplome_x_exp
   15. diplome_x_note
   16. exp_x_note
   17. score_composite
   18. exp_elevee_pour_age
   19. salaire_eleve
   20. note_elevee
   21. diplome_eleve
   22. est_senior
   23. profil_atypique

Statistiques descriptives des nouvelles variables numériques :
          annee      mois  trimestre  jour_semaine  semaine_annee  \
count  20000.00  20000.00   20000.00      20000.00       20000.00   
mean    2012.00      6.51       2.50          3.01          26.45   
std        1.41      3.46       1.12          2.00          15.13   
min     2010.00      1.00       1.00          0.00           1.00   
25%     201

In [140]:
# =============================================================================
# BILAN FINAL
# =============================================================================

print("\n" + "="*80)
print("BILAN FINAL DU FEATURE ENGINEERING")
print("="*80)

print(f"\nFeature Engineering terminé avec succès !\n")
print(f"   - 24 nouvelles features créées par base")
print(f"   - 3 bases enrichies sauvegardées avec suffixe '_features.csv'")
print(f"\nFichiers générés :")
print(f"   - ../data/df_dropna_features.csv")
print(f"   - ../data/df_median_mode_features.csv")
print(f"   - ../data/df_knn_features.csv")

print(f"\nCatégories de features :")
print(f"   [1] Temporelles (5) : mois, trimestre, année, jour_semaine, semaine_annee")
print(f"   [2] Ratios (4) : exp_age_ratio, salaire_par_exp, etc.")
print(f"   [3] Catégories (4) : categorie_age, niveau_experience, etc.")
print(f"   [4] Interactions (5) : diplome_x_exp, diplome_x_note, etc.")
print(f"   [5] Flags (6) : est_senior, salaire_eleve, etc.")


BILAN FINAL DU FEATURE ENGINEERING

Feature Engineering terminé avec succès !

   - 24 nouvelles features créées par base
   - 3 bases enrichies sauvegardées avec suffixe '_features.csv'

Fichiers générés :
   - ../data/df_dropna_features.csv
   - ../data/df_median_mode_features.csv
   - ../data/df_knn_features.csv

Catégories de features :
   [1] Temporelles (5) : mois, trimestre, année, jour_semaine, semaine_annee
   [2] Ratios (4) : exp_age_ratio, salaire_par_exp, etc.
   [3] Catégories (4) : categorie_age, niveau_experience, etc.
   [4] Interactions (5) : diplome_x_exp, diplome_x_note, etc.
   [5] Flags (6) : est_senior, salaire_eleve, etc.


#### SECTION 3 : ENCODAGE DES VARIABLES CATEGORIELLES

Objectif : Transformer les variables catégorielles en format numérique
Stratégie : 
- Label Encoding pour variables ordinales et binaires (diplôme, sexe, dispo)
- One-Hot Encoding pour variables nominales (cheveux, spécialité)
- Encodage cyclique pour variables temporelles (mois, jour_semaine)

In [141]:
# =============================================================================
# FONCTION D'ENCODAGE
# =============================================================================

def encoder_variables(df_input, nom_base):
    """
    Encode toutes les variables catégorielles selon leur type
    
    Paramètres :
    - df_input : DataFrame à encoder
    - nom_base : nom du fichier pour l'affichage
    
    Retourne :
    - DataFrame avec variables encodées
    """
    
    print(f"\n{'='*80}")
    print(f"Traitement de : {nom_base}")
    print(f"{'='*80}")
    
    df = df_input.copy()
    
    print(f"\nNombre de colonnes AVANT encodage : {len(df.columns)}")
    
    # -------------------------------------------------------------------------
    # 1. LABEL ENCODING - VARIABLES ORDINALES
    # -------------------------------------------------------------------------
    
    print("\n[1/4] Label Encoding pour variables ORDINALES...")
    
    # DIPLÔME (hiérarchie claire : bac < licence < master < doctorat)
    diplome_mapping = {
        'bac': 0,
        'licence': 1,
        'master': 2,
        'doctorat': 3
    }
    
    if 'diplome' in df.columns:
        df['diplome_encoded'] = df['diplome'].map(diplome_mapping)
        print(f"   - diplome : encodé selon hiérarchie (bac=0, licence=1, master=2, doctorat=3)")
        print(f"     Répartition : {df['diplome'].value_counts().to_dict()}")
    
    # NIVEAU_EXPERIENCE (si créé en Feature Engineering)
    if 'niveau_experience' in df.columns:
        niveau_exp_mapping = {
            'Debutant': 0,
            'Intermediaire': 1,
            'Experimente': 2,
            'Expert': 3
        }
        df['niveau_experience_encoded'] = df['niveau_experience'].map(niveau_exp_mapping)
        print(f"   - niveau_experience : encodé selon hiérarchie (Debutant=0, Expert=3)")
    
    # CATEGORIE_AGE (si créée en Feature Engineering)
    if 'categorie_age' in df.columns:
        categorie_age_mapping = {
            'Jeune': 0,
            'Junior': 1,
            'Confirme': 2,
            'Senior': 3
        }
        df['categorie_age_encoded'] = df['categorie_age'].map(categorie_age_mapping)
        print(f"   - categorie_age : encodé selon hiérarchie (Jeune=0, Senior=3)")
    
    # TRANCHE_SALAIRE (si créée en Feature Engineering)
    if 'tranche_salaire' in df.columns:
        tranche_salaire_mapping = {
            'Bas': 0,
            'Moyen': 1,
            'Eleve': 2,
            'Tres_eleve': 3
        }
        df['tranche_salaire_encoded'] = df['tranche_salaire'].map(tranche_salaire_mapping)
        print(f"   - tranche_salaire : encodé selon hiérarchie (Bas=0, Tres_eleve=3)")
    
    # NIVEAU_NOTE (si créé en Feature Engineering)
    if 'niveau_note' in df.columns:
        niveau_note_mapping = {
            'Faible': 0,
            'Moyen': 1,
            'Bon': 2,
            'Excellent': 3
        }
        df['niveau_note_encoded'] = df['niveau_note'].map(niveau_note_mapping)
        print(f"   - niveau_note : encodé selon hiérarchie (Faible=0, Excellent=3)")
    
    # -------------------------------------------------------------------------
    # 2. LABEL ENCODING - VARIABLES BINAIRES
    # -------------------------------------------------------------------------
    
    print("\n[2/4] Label Encoding pour variables BINAIRES...")
    
    # SEXE
    if 'sexe' in df.columns:
        sexe_mapping = {'F': 0, 'M': 1}
        df['sexe_encoded'] = df['sexe'].map(sexe_mapping)
        print(f"   - sexe : encodé (F=0, M=1)")
        print(f"     Répartition : {df['sexe'].value_counts().to_dict()}")
    
    # DISPO
    if 'dispo' in df.columns:
        dispo_mapping = {'non': 0, 'oui': 1}
        df['dispo_encoded'] = df['dispo'].map(dispo_mapping)
        print(f"   - dispo : encodé (non=0, oui=1)")
        print(f"     Répartition : {df['dispo'].value_counts().to_dict()}")
    
    # -------------------------------------------------------------------------
    # 3. ONE-HOT ENCODING - VARIABLES NOMINALES
    # -------------------------------------------------------------------------
    
    print("\n[3/4] One-Hot Encoding pour variables NOMINALES...")
    
    # CHEVEUX (pas de hiérarchie entre couleurs)
    if 'cheveux' in df.columns:
        nb_modalites = df['cheveux'].nunique()
        df = pd.get_dummies(df, columns=['cheveux'], prefix='cheveux', drop_first=True)
        print(f"   - cheveux : {nb_modalites} modalités -> {nb_modalites-1} colonnes binaires")
        print(f"     Colonnes créées : {[col for col in df.columns if col.startswith('cheveux_')]}")
    
    # SPECIALITE (pas de hiérarchie entre spécialités)
    if 'specialite' in df.columns:
        nb_modalites = df['specialite'].nunique()
        print(f"   - specialite : {nb_modalites} modalités détectées")
        
        # Si trop de modalités (> 15), regrouper les rares en "Autre"
        if nb_modalites > 15:
            print(f"     ATTENTION : Trop de modalités ({nb_modalites})")
            print(f"     Regroupement des spécialités rares en 'Autre'...")
            
            # Garder les 15 spécialités les plus fréquentes
            top_specialites = df['specialite'].value_counts().head(15).index.tolist()
            df['specialite'] = df['specialite'].apply(
                lambda x: x if x in top_specialites else 'Autre'
            )
            nb_modalites = df['specialite'].nunique()
            print(f"     Après regroupement : {nb_modalites} modalités")
        
        df = pd.get_dummies(df, columns=['specialite'], prefix='spec', drop_first=True)
        print(f"     Colonnes créées : {nb_modalites-1}")
    
    # -------------------------------------------------------------------------
    # 4. ENCODAGE CYCLIQUE - VARIABLES TEMPORELLES
    # -------------------------------------------------------------------------
    
    print("\n[4/4] Encodage CYCLIQUE pour variables TEMPORELLES...")
    
    # MOIS (cyclique : décembre proche de janvier)
    if 'mois' in df.columns:
        df['mois_sin'] = np.sin(2 * np.pi * df['mois'] / 12)
        df['mois_cos'] = np.cos(2 * np.pi * df['mois'] / 12)
        print(f"   - mois : encodage cyclique (mois_sin, mois_cos)")
        print(f"     Justification : décembre (12) est proche de janvier (1)")
    
    # JOUR_SEMAINE (cyclique : dimanche proche de lundi)
    if 'jour_semaine' in df.columns:
        df['jour_semaine_sin'] = np.sin(2 * np.pi * df['jour_semaine'] / 7)
        df['jour_semaine_cos'] = np.cos(2 * np.pi * df['jour_semaine'] / 7)
        print(f"   - jour_semaine : encodage cyclique (jour_semaine_sin, jour_semaine_cos)")
        print(f"     Justification : dimanche (6) est proche de lundi (0)")
    
    # TRIMESTRE (ordinal simple, pas besoin de cyclique)
    if 'trimestre' in df.columns:
        print(f"   - trimestre : conservé tel quel (1, 2, 3, 4)")
        print(f"     Note : déjà numérique et ordinal")
    
    # SEMAINE_ANNEE (cyclique : semaine 52 proche de semaine 1)
    if 'semaine_annee' in df.columns:
        df['semaine_annee_sin'] = np.sin(2 * np.pi * df['semaine_annee'] / 52)
        df['semaine_annee_cos'] = np.cos(2 * np.pi * df['semaine_annee'] / 52)
        print(f"   - semaine_annee : encodage cyclique (semaine_annee_sin, semaine_annee_cos)")
    
    # -------------------------------------------------------------------------
    # NETTOYAGE - SUPPRESSION DES COLONNES ORIGINALES ENCODÉES
    # -------------------------------------------------------------------------
    
    print("\n[5/4] Nettoyage : suppression des colonnes originales encodées...")
    
    colonnes_a_supprimer = []
    
    # Supprimer les versions textuelles si versions encodées existent
    if 'diplome_encoded' in df.columns:
        colonnes_a_supprimer.append('diplome')
    if 'sexe_encoded' in df.columns:
        colonnes_a_supprimer.append('sexe')
    if 'dispo_encoded' in df.columns:
        colonnes_a_supprimer.append('dispo')
    if 'niveau_experience_encoded' in df.columns:
        colonnes_a_supprimer.append('niveau_experience')
    if 'categorie_age_encoded' in df.columns:
        colonnes_a_supprimer.append('categorie_age')
    if 'tranche_salaire_encoded' in df.columns:
        colonnes_a_supprimer.append('tranche_salaire')
    if 'niveau_note_encoded' in df.columns:
        colonnes_a_supprimer.append('niveau_note')
    
    # Supprimer les colonnes temporelles originales si encodées cycliquement
    if 'mois_sin' in df.columns and 'mois' in df.columns:
        colonnes_a_supprimer.append('mois')
    if 'jour_semaine_sin' in df.columns and 'jour_semaine' in df.columns:
        colonnes_a_supprimer.append('jour_semaine')
    if 'semaine_annee_sin' in df.columns and 'semaine_annee' in df.columns:
        colonnes_a_supprimer.append('semaine_annee')
    
    # Supprimer la colonne date (déjà extraite en variables temporelles)
    if 'date' in df.columns:
        colonnes_a_supprimer.append('date')
    
    # Supprimer la colonne index (non prédictive)
    if 'index' in df.columns:
        colonnes_a_supprimer.append('index')
    
    # Supprimer diplome_num si diplome_encoded existe (doublon)
    if 'diplome_encoded' in df.columns and 'diplome_num' in df.columns:
        colonnes_a_supprimer.append('diplome_num')
    
    # Filtrer les colonnes qui existent réellement
    colonnes_a_supprimer = [col for col in colonnes_a_supprimer if col in df.columns]
    
    if len(colonnes_a_supprimer) > 0:
        df = df.drop(columns=colonnes_a_supprimer)
        print(f"   - {len(colonnes_a_supprimer)} colonnes supprimées : {colonnes_a_supprimer}")
    else:
        print(f"   - Aucune colonne à supprimer")
    
    # -------------------------------------------------------------------------
    # RÉCAPITULATIF
    # -------------------------------------------------------------------------
    
    print(f"\n{'='*80}")
    print(f"RÉCAPITULATIF - {nom_base}")
    print(f"{'='*80}")
    
    print(f"\nNombre de colonnes APRÈS encodage : {len(df.columns)}")
    
    # Compter les types de colonnes
    colonnes_numeriques = df.select_dtypes(include=[np.number]).columns.tolist()
    colonnes_categoriques = df.select_dtypes(exclude=[np.number]).columns.tolist()
    
    print(f"\nTypes de colonnes :")
    print(f"   - Numériques : {len(colonnes_numeriques)}")
    print(f"   - Catégorielles restantes : {len(colonnes_categoriques)}")
    
    if len(colonnes_categoriques) > 0:
        print(f"     ATTENTION : Colonnes catégorielles non encodées : {colonnes_categoriques}")
    
    # Vérifier s'il reste des valeurs manquantes
    nb_missing = df.isnull().sum().sum()
    print(f"\nValeurs manquantes : {nb_missing}")
    
    if nb_missing > 0:
        print(f"   ATTENTION : Colonnes avec NaN :")
        print(df.isnull().sum()[df.isnull().sum() > 0])
    
    return df

In [142]:


# =============================================================================
# APPLICATION SUR LES 3 BASES DE DONNÉES
# =============================================================================

# Chemin du dossier data
data_path = '../data'

bases = [
    'df_dropna_features.csv',
    'df_median_mode_features.csv', 
    'df_knn_features.csv'
]

for nom_base in bases:
    try:
        # Chemin complet
        chemin_fichier = os.path.join(data_path, nom_base)
        
        # Charger la base
        print(f"\nChargement de {nom_base}...")
        df = pd.read_csv(chemin_fichier)
        
        # Appliquer l'encodage
        df_encoded = encoder_variables(df, nom_base)
        
        # Sauvegarder avec un nouveau nom
        nom_output = nom_base.replace('_features.csv', '_encoded.csv')
        chemin_output = os.path.join(data_path, nom_output)
        df_encoded.to_csv(chemin_output, index=False)
        
        print(f"\nBase encodée sauvegardée : {chemin_output}")
        
    except FileNotFoundError:
        print(f"\nATTENTION : Fichier non trouvé : {chemin_fichier}")
        continue
    except Exception as e:
        print(f"\nERREUR lors du traitement de {nom_base} : {e}")
        import traceback
        traceback.print_exc()
        continue


Chargement de df_dropna_features.csv...

Traitement de : df_dropna_features.csv

Nombre de colonnes AVANT encodage : 35

[1/4] Label Encoding pour variables ORDINALES...
   - diplome : encodé selon hiérarchie (bac=0, licence=1, master=2, doctorat=3)
     Répartition : {'master': 7176, 'licence': 7061, 'doctorat': 2440, 'bac': 2344}
   - niveau_experience : encodé selon hiérarchie (Debutant=0, Expert=3)
   - categorie_age : encodé selon hiérarchie (Jeune=0, Senior=3)
   - tranche_salaire : encodé selon hiérarchie (Bas=0, Tres_eleve=3)
   - niveau_note : encodé selon hiérarchie (Faible=0, Excellent=3)

[2/4] Label Encoding pour variables BINAIRES...
   - sexe : encodé (F=0, M=1)
     Répartition : {'M': 11355, 'F': 7666}
   - dispo : encodé (non=0, oui=1)
     Répartition : {'non': 11332, 'oui': 7689}

[3/4] One-Hot Encoding pour variables NOMINALES...
   - cheveux : 4 modalités -> 3 colonnes binaires
     Colonnes créées : ['cheveux_brun', 'cheveux_chatain', 'cheveux_roux']
   - specia

In [144]:

# =============================================================================
# VÉRIFICATION FINALE
# =============================================================================

print("\n" + "="*80)
print("VÉRIFICATION FINALE")
print("="*80)

# Charger une des bases encodées pour vérification
try:
    chemin_verif = os.path.join(data_path, 'df_median_mode_encoded.csv')
    df_verif = pd.read_csv(chemin_verif)
    
    print(f"\nVérification sur df_median_mode_encoded.csv :")
    print(f"\nDimensions : {df_verif.shape}")
    print(f"   - Lignes : {df_verif.shape[0]}")
    print(f"   - Colonnes : {df_verif.shape[1]}")
    
    print(f"\nTypes de données :")
    print(df_verif.dtypes.value_counts())
    
    print(f"\nPremières colonnes (aperçu) :")
    print(df_verif.columns.tolist()[:20])
    
    print(f"\nExemple de données encodées :")
    colonnes_affichage = ['age', 'exp', 'salaire', 'note', 'diplome_encoded', 
                         'sexe_encoded', 'dispo_encoded', 'embauche']
    colonnes_disponibles = [col for col in colonnes_affichage if col in df_verif.columns]
    print(df_verif[colonnes_disponibles].head(10))
    
    # Vérifier qu'il ne reste aucune colonne catégorielle non encodée
    colonnes_object = df_verif.select_dtypes(include=['object']).columns.tolist()
    if len(colonnes_object) > 0:
        print(f"\nATTENTION : Colonnes de type 'object' (non numériques) restantes :")
        print(colonnes_object)
    else:
        print(f"\nSUCCES : Toutes les variables sont numériques !")

except Exception as e:
    print(f"\nImpossible de charger le fichier de vérification : {e}")




VÉRIFICATION FINALE

Vérification sur df_median_mode_encoded.csv :

Dimensions : (20000, 39)
   - Lignes : 20000
   - Colonnes : 39

Types de données :
float64    18
int64      15
bool        6
Name: count, dtype: int64

Premières colonnes (aperçu) :
['age', 'exp', 'salaire', 'note', 'embauche', 'annee', 'trimestre', 'exp_age_ratio', 'salaire_par_exp', 'ecart_exp_max', 'diplome_x_exp', 'diplome_x_note', 'exp_x_note', 'score_composite', 'exp_elevee_pour_age', 'salaire_eleve', 'note_elevee', 'diplome_eleve', 'est_senior', 'profil_atypique']

Exemple de données encodées :
    age   exp  salaire   note  diplome_encoded  sexe_encoded  dispo_encoded  \
0  25.0   9.0  26803.0  97.08                1             0              0   
1  35.0  13.0  38166.0  63.86                1             1              0   
2  29.0  13.0  35207.0  78.50                1             1              0   
3  35.0  12.0  32442.0  45.09                1             1              0   
4  35.0   6.0  28533.0  81.9

In [145]:
# =============================================================================
# BILAN FINAL
# =============================================================================

print("\n" + "="*80)
print("BILAN FINAL DE L'ENCODAGE")
print("="*80)

print(f"\nEncodage terminé avec succès !\n")

print(f"Fichiers générés :")
print(f"   - ../data/df_dropna_encoded.csv")
print(f"   - ../data/df_median_mode_encoded.csv")
print(f"   - ../data/df_knn_encoded.csv")

print(f"\nMéthodes d'encodage appliquées :")
print(f"   [1] Label Encoding (ordinales) :")
print(f"       - diplome (bac=0, licence=1, master=2, doctorat=3)")
print(f"       - niveau_experience, categorie_age, tranche_salaire, niveau_note")
print(f"\n   [2] Label Encoding (binaires) :")
print(f"       - sexe (F=0, M=1)")
print(f"       - dispo (non=0, oui=1)")
print(f"\n   [3] One-Hot Encoding (nominales) :")
print(f"       - cheveux (drop_first=True)")
print(f"       - specialite (avec regroupement si > 15 modalités)")
print(f"\n   [4] Encodage cyclique (temporelles) :")
print(f"       - mois (mois_sin, mois_cos)")
print(f"       - jour_semaine (jour_semaine_sin, jour_semaine_cos)")
print(f"       - semaine_annee (semaine_annee_sin, semaine_annee_cos)")

print(f"\nColonnes supprimées :")
print(f"   - Versions textuelles des variables encodées")
print(f"   - date (remplacée par variables temporelles)")
print(f"   - index (non prédictive)")

print("\n" + "="*80)
print("PHASE 2.3 TERMINÉE - Prêt pour la normalisation/standardisation !")
print("="*80)


BILAN FINAL DE L'ENCODAGE

Encodage terminé avec succès !

Fichiers générés :
   - ../data/df_dropna_encoded.csv
   - ../data/df_median_mode_encoded.csv
   - ../data/df_knn_encoded.csv

Méthodes d'encodage appliquées :
   [1] Label Encoding (ordinales) :
       - diplome (bac=0, licence=1, master=2, doctorat=3)
       - niveau_experience, categorie_age, tranche_salaire, niveau_note

   [2] Label Encoding (binaires) :
       - sexe (F=0, M=1)
       - dispo (non=0, oui=1)

   [3] One-Hot Encoding (nominales) :
       - cheveux (drop_first=True)
       - specialite (avec regroupement si > 15 modalités)

   [4] Encodage cyclique (temporelles) :
       - mois (mois_sin, mois_cos)
       - jour_semaine (jour_semaine_sin, jour_semaine_cos)
       - semaine_annee (semaine_annee_sin, semaine_annee_cos)

Colonnes supprimées :
   - Versions textuelles des variables encodées
   - date (remplacée par variables temporelles)
   - index (non prédictive)

PHASE 2.3 TERMINÉE - Prêt pour la normalisati

#### SECTION 5 : NORMALISATION / STANDARDISATION

Objectif : Standardiser les variables numériques pour les algorithmes sensibles
Méthode : StandardScaler (moyenne=0, écart-type=1)

Note : Cette normalisation sera appliquée GLOBALEMENT sur les 3 bases.
       Lors de la modélisation, il faudra refaire un fit() sur train uniquement.


In [150]:
# =============================================================================
# FONCTION DE NORMALISATION
# =============================================================================

def normaliser_variables(df_input, nom_base):
    """
    Standardise les variables numériques (moyenne=0, écart-type=1)
    
    Paramètres :
    - df_input : DataFrame à normaliser
    - nom_base : nom du fichier pour l'affichage
    
    Retourne :
    - DataFrame normalisé
    - Scaler ajusté (pour réutilisation)
    """
    
    print(f"\n{'='*80}")
    print(f"Traitement de : {nom_base}")
    print(f"{'='*80}")
    
    df = df_input.copy()
    
    # -------------------------------------------------------------------------
    # 1. IDENTIFICATION DES VARIABLES À NORMALISER
    # -------------------------------------------------------------------------
    
    print("\n[1/3] Identification des variables à normaliser...")
    
    # UNIQUEMENT LE SALAIRE sera normalisé
    variables_a_normaliser = ['salaire']
    
    # Vérifier que la variable existe
    if 'salaire' not in df.columns:
        print("   ERREUR : La variable 'salaire' n'existe pas dans le dataset")
        return df, None
    
    print(f"\n   Variable à normaliser : salaire")
    print(f"   Justification : Seule variable avec une échelle très différente des autres")
    
    # -------------------------------------------------------------------------
    # 2. APPLICATION DU STANDARDSCALER
    # -------------------------------------------------------------------------
    
    print(f"\n[2/3] Application de StandardScaler...")
    
    if len(variables_a_normaliser) == 0:
        print("   ATTENTION : Aucune variable à normaliser détectée")
        return df, None
    
    # Créer le scaler
    scaler = StandardScaler()
    
    # Afficher les statistiques AVANT normalisation
    print(f"\n   Statistiques du SALAIRE AVANT normalisation :")
    stats_avant = df[['salaire']].describe().loc[['mean', 'std', 'min', 'max']]
    print(stats_avant.round(2))
    
    # Appliquer la normalisation
    df[variables_a_normaliser] = scaler.fit_transform(df[variables_a_normaliser])
    
    # Afficher les statistiques APRÈS normalisation
    print(f"\n   Statistiques du SALAIRE APRÈS normalisation :")
    stats_apres = df[['salaire']].describe().loc[['mean', 'std', 'min', 'max']]
    print(stats_apres.round(2))
    
    print(f"\n   StandardScaler appliqué avec succès")
    print(f"   - Moyenne des variables normalisées : ~0")
    print(f"   - Écart-type des variables normalisées : ~1")
    
    # -------------------------------------------------------------------------
    # 3. VÉRIFICATION
    # -------------------------------------------------------------------------
    
    print(f"\n[3/3] Vérification post-normalisation...")
    
    # Vérifier qu'il n'y a pas de valeurs infinies ou NaN créées
    nb_inf = np.isinf(df['salaire']).sum()
    nb_nan = df['salaire'].isnull().sum()
    
    print(f"\n   Valeurs infinies créées : {nb_inf}")
    print(f"   Valeurs NaN créées : {nb_nan}")
    
    if nb_inf > 0 or nb_nan > 0:
        print(f"   ATTENTION : Problème détecté lors de la normalisation")
    else:
        print(f"   SUCCES : Aucune valeur aberrante créée")
    
    # Vérifier que les autres variables n'ont pas été modifiées
    print(f"\n   Variables NON normalisées (conservées telles quelles) :")
    variables_non_modifiees = ['age', 'exp', 'note']
    variables_disponibles = [col for col in variables_non_modifiees if col in df.columns]
    if len(variables_disponibles) > 0:
        print(df[variables_disponibles].describe().loc[['mean', 'min', 'max']].round(2))
    
    # -------------------------------------------------------------------------
    # RÉCAPITULATIF
    # -------------------------------------------------------------------------
    
    print(f"\n{'='*80}")
    print(f"RÉCAPITULATIF - {nom_base}")
    print(f"{'='*80}")
    
    print(f"\n   Variable normalisée : salaire")
    print(f"   Toutes les autres variables : conservées telles quelles")
    print(f"   Méthode : StandardScaler (moyenne=0, écart-type=1)")
    
    return df, scaler


In [151]:
# =============================================================================
# APPLICATION SUR LES 3 BASES DE DONNÉES
# =============================================================================

print("\n" + "="*80)
print("APPLICATION SUR LES 3 BASES DE DONNÉES")
print("="*80)

# Chemin du dossier data
data_path = '../data'

bases = [
    'df_dropna_encoded.csv',
    'df_median_mode_encoded.csv', 
    'df_knn_encoded.csv'
]

scalers = {}  # Pour sauvegarder les scalers

for nom_base in bases:
    try:
        # Chemin complet
        chemin_fichier = os.path.join(data_path, nom_base)
        
        # Charger la base
        print(f"\nChargement de {nom_base}...")
        df = pd.read_csv(chemin_fichier)
        
        # Appliquer la normalisation
        df_normalized, scaler = normaliser_variables(df, nom_base)
        
        # Sauvegarder le dataset normalisé
        nom_output = nom_base.replace('_encoded.csv', '_normalized.csv')
        chemin_output = os.path.join(data_path, nom_output)
        df_normalized.to_csv(chemin_output, index=False)
        
        print(f"\nBase normalisée sauvegardée : {chemin_output}")
        
        # Sauvegarder le scaler
        if scaler is not None:
            nom_scaler = nom_base.replace('_encoded.csv', '_scaler.pkl')
            chemin_scaler = os.path.join(data_path, nom_scaler)
            joblib.dump(scaler, chemin_scaler)
            scalers[nom_base] = scaler
            print(f"Scaler sauvegardé : {chemin_scaler}")
        
    except FileNotFoundError:
        print(f"\nATTENTION : Fichier non trouvé : {chemin_fichier}")
        continue
    except Exception as e:
        print(f"\nERREUR lors du traitement de {nom_base} : {e}")
        import traceback
        traceback.print_exc()
        continue


APPLICATION SUR LES 3 BASES DE DONNÉES

Chargement de df_dropna_encoded.csv...

Traitement de : df_dropna_encoded.csv

[1/3] Identification des variables à normaliser...

   Variable à normaliser : salaire
   Justification : Seule variable avec une échelle très différente des autres

[2/3] Application de StandardScaler...

   Statistiques du SALAIRE AVANT normalisation :
       salaire
mean  34966.88
std    5005.21
min   14128.00
max   53977.00

   Statistiques du SALAIRE APRÈS normalisation :
      salaire
mean     0.00
std      1.00
min     -4.16
max      3.80

   StandardScaler appliqué avec succès
   - Moyenne des variables normalisées : ~0
   - Écart-type des variables normalisées : ~1

[3/3] Vérification post-normalisation...

   Valeurs infinies créées : 0
   Valeurs NaN créées : 0
   SUCCES : Aucune valeur aberrante créée

   Variables NON normalisées (conservées telles quelles) :
       age   exp    note
mean  35.0   9.5   72.97
min    0.0   0.0    8.68
max   72.0  23.0  100.